# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 13 2023 3:06PM,259191,15,VT80184993,"Virtus Pharmaceuticals, LLC",Executing
1,Mar 13 2023 3:06PM,259191,15,VT80184994,"Virtus Pharmaceuticals, LLC",Released
2,Mar 13 2023 3:06PM,259191,15,VT80184995,"Virtus Pharmaceuticals, LLC",Released
3,Mar 13 2023 3:06PM,259191,15,VT80184996,"Virtus Pharmaceuticals, LLC",Executing
4,Mar 13 2023 3:06PM,259191,15,VT80184997,"Virtus Pharmaceuticals, LLC",Executing
5,Mar 13 2023 3:06PM,259191,15,VT80184998,"Virtus Pharmaceuticals, LLC",Released
6,Mar 13 2023 3:06PM,259191,15,VT80184999,"Virtus Pharmaceuticals, LLC",Executing
7,Mar 13 2023 3:06PM,259191,15,VT80185000,"Virtus Pharmaceuticals, LLC",Executing
8,Mar 13 2023 3:06PM,259191,15,VT80185001,"Virtus Pharmaceuticals, LLC",Released
9,Mar 13 2023 3:06PM,259191,15,VT80185002,"Virtus Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,259200,Released,2
16,259201,Released,1
17,259202,Released,7
18,259203,Released,1
19,259205,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
259200,NaN,NaN,2.0
259201,NaN,NaN,1.0
259202,NaN,NaN,7.0
259203,NaN,NaN,1.0
259205,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259150,4.0,7.0,3.0
259152,0.0,1.0,0.0
259168,0.0,1.0,0.0
259174,2.0,2.0,5.0
259175,12.0,9.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
259150,4,7,3
259152,0,1,0
259168,0,1,0
259174,2,2,5
259175,12,9,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259150,4,7,3
1,259152,0,1,0
2,259168,0,1,0
3,259174,2,2,5
4,259175,12,9,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,259150,4,7,3
1,259152,,1,
2,259168,,1,
3,259174,2,2,5
4,259175,12,9,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 13 2023 3:06PM,259191,15,"Virtus Pharmaceuticals, LLC"
11,Mar 13 2023 2:55PM,259205,10,"Citieffe, Inc."
13,Mar 13 2023 2:44PM,259203,10,Hush Hush
14,Mar 13 2023 2:42PM,259202,10,Bio-PRF
21,Mar 13 2023 2:34PM,259201,10,"Snap Medical Industries, LLC"
22,Mar 13 2023 2:28PM,259200,10,Emerginnova
24,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc."
25,Mar 13 2023 1:36PM,259187,10,Emerginnova
27,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC"
51,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 13 2023 3:06PM,259191,15,"Virtus Pharmaceuticals, LLC",,6,5
1,Mar 13 2023 2:55PM,259205,10,"Citieffe, Inc.",,,2
2,Mar 13 2023 2:44PM,259203,10,Hush Hush,,,1
3,Mar 13 2023 2:42PM,259202,10,Bio-PRF,,,7
4,Mar 13 2023 2:34PM,259201,10,"Snap Medical Industries, LLC",,,1
5,Mar 13 2023 2:28PM,259200,10,Emerginnova,,,2
6,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",,,1
7,Mar 13 2023 1:36PM,259187,10,Emerginnova,,,2
8,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",12,9,3
9,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",2,2,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 3:06PM,259191,15,"Virtus Pharmaceuticals, LLC",5,6,
1,Mar 13 2023 2:55PM,259205,10,"Citieffe, Inc.",2,,
2,Mar 13 2023 2:44PM,259203,10,Hush Hush,1,,
3,Mar 13 2023 2:42PM,259202,10,Bio-PRF,7,,
4,Mar 13 2023 2:34PM,259201,10,"Snap Medical Industries, LLC",1,,
5,Mar 13 2023 2:28PM,259200,10,Emerginnova,2,,
6,Mar 13 2023 1:54PM,259198,21,"OK Capsule, Inc.",1,,
7,Mar 13 2023 1:36PM,259187,10,Emerginnova,2,,
8,Mar 13 2023 11:40AM,259175,15,"Brookfield Pharmaceuticals, LLC",3,9,12
9,Mar 13 2023 11:33AM,259174,10,"Snap Medical Industries, LLC",5,2,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 3:06PM,259191,15,"Virtus Pharmaceuticals, LLC",5,6,
1,Mar 13 2023 2:55PM,259205,10,"Citieffe, Inc.",2,,
2,Mar 13 2023 2:44PM,259203,10,Hush Hush,1,,
3,Mar 13 2023 2:42PM,259202,10,Bio-PRF,7,,
4,Mar 13 2023 2:34PM,259201,10,"Snap Medical Industries, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 3:06PM,259191,15,"Virtus Pharmaceuticals, LLC",5.0,6.0,NaN
1,Mar 13 2023 2:55PM,259205,10,"Citieffe, Inc.",2.0,NaN,NaN
2,Mar 13 2023 2:44PM,259203,10,Hush Hush,1.0,NaN,NaN
3,Mar 13 2023 2:42PM,259202,10,Bio-PRF,7.0,NaN,NaN
4,Mar 13 2023 2:34PM,259201,10,"Snap Medical Industries, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 13 2023 3:06PM,259191,15,"Virtus Pharmaceuticals, LLC",5.0,6.0,0.0
1,Mar 13 2023 2:55PM,259205,10,"Citieffe, Inc.",2.0,0.0,0.0
2,Mar 13 2023 2:44PM,259203,10,Hush Hush,1.0,0.0,0.0
3,Mar 13 2023 2:42PM,259202,10,Bio-PRF,7.0,0.0,0.0
4,Mar 13 2023 2:34PM,259201,10,"Snap Medical Industries, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2073522,23.0,9.0,6.0
15,518366,8.0,15.0,12.0
18,259168,0.0,1.0,0.0
19,259152,0.0,1.0,0.0
21,259198,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2073522,23.0,9.0,6.0
1,15,518366,8.0,15.0,12.0
2,18,259168,0.0,1.0,0.0
3,19,259152,0.0,1.0,0.0
4,21,259198,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,23.0,9.0,6.0
1,15,8.0,15.0,12.0
2,18,0.0,1.0,0.0
3,19,0.0,1.0,0.0
4,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,23.0
1,15,Released,8.0
2,18,Released,0.0
3,19,Released,0.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21
Status,,,,,
Completed,6.0,12.0,0.0,0.0,0.0
Executing,9.0,15.0,1.0,1.0,0.0
Released,23.0,8.0,0.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21
0,Completed,6.0,12.0,0.0,0.0,0.0
1,Executing,9.0,15.0,1.0,1.0,0.0
2,Released,23.0,8.0,0.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21
0,Completed,6.0,12.0,0.0,0.0,0.0
1,Executing,9.0,15.0,1.0,1.0,0.0
2,Released,23.0,8.0,0.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()